In [ ]:
# Ignorar sklearn warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='skimage')

# Librería de ámbito matemático, científico y de ingeniería. En nuestro caso, solo usamos la convolución.
import scipy.io

# Interfaz para HDF5. Solo necesario para cargar las anotaciones del dataset.
import hdf5storage

# Librería para la computación científica. En nuestro caso, la usamos por la representación de datos y hacer calculas de forma
# rápida.
import numpy as np

# Librerías de visión por computador
# OpenCV: Generalista.
# Skimage: Generalista.
# Dlib: Menos funcionalidades, muy rápido e implementaciones muy interesantes.
import dlib
import cv2
from skimage import io
from skimage import color
from skimage import filters
from skimage import util
from skimage import transform
from skimage import feature

# Librerías para gráficos en 2d/3d. En nuestro caso para mostrar las imágenes y gráficos en general.
from matplotlib import pyplot as plt
from IPython.display import HTML
from IPython.display import Video

# Necesario para visualizar los resultados en la misma página.
%matplotlib inline

# Introducción

Para comprender mejor las explicaciones de la presentación procederemos a realizar las siguientes tareas:

* __Transformaciones afines__: Aplicaremos diferentes transformaciones afines y las usaremos en un ejemplo práctico.

* __Proyectos__: En varios ejemplos enseño las diferentes utilidades de las transformaciones afines



# Transformaciones afines

## Translación 

El caso más sencillo es el de la translación, si recordamos de la parte teórica la matriz tiene la siguiente forma:

![Traslacion](http://opencv-python-tutroals.readthedocs.io/en/latest/_images/math/22fe551f03b8e94f1a7a75731a660f0163030540.png)

In [ ]:
# Cargamos la imagen
img = io.imread("resources/session1/altran.jpg")
filas, columnas, canales = img.shape

# Definimos la matriz de traslación
M = np.float32([[1,0,100],[0,1,50]])
# Aplicamos la transformación
dst = cv2.warpAffine(img,M,(columnas,filas))

# Definimos la matriz de traslación
M2 = np.float32([[1,0,5],[0,1,20]])
# Aplicamos la transformación
dst2 = cv2.warpAffine(img,M2,(columnas,filas))

# Mostramos la imagen
plt.figure(figsize=(20, 10))
plt.subplot("131")
plt.imshow(img)
plt.subplot("132")
plt.imshow(dst)
plt.subplot("133")
plt.imshow(dst2)

## Rotación
En este caso la matriz tiene la siguiente forma:

![Imagen](http://opencv-python-tutroals.readthedocs.io/en/latest/_images/math/f3a6bed945808a1f3a9df71b260f68f8e653af95.png)

pero en el caso de opencv se ha fijado de forma que el centro siga en el mismo lugar de la siguiente forma:

![Opencv](http://opencv-python-tutroals.readthedocs.io/en/latest/_images/math/91ff2b9b1db0760f4764631010749e594cdf5f5f.png)

In [ ]:
M = cv2.getRotationMatrix2D((columnas/2,filas/2),25,1)
dst = cv2.warpAffine(img,M,(columnas * 2,filas * 2))

# Mostramos la imagen
plt.figure(figsize=(20, 10))
plt.subplot("121")
plt.imshow(img)
plt.subplot("122")
plt.imshow(dst)

## Transformación afín

En la transformación afín, todas las líneas paralelas en la imagen original seguirán siendo paralelas en la imagen de salida. Para calcular la matriz de transformación, necesitamos localizar tres puntos en la imagen de entrada y sus ubicaciones también en la imagen de salida.


In [ ]:
pts1 = np.float32([[50,50],[200,50],[50,200]])
pts2 = np.float32([[40,60],[200,50],[75,225]])

M = cv2.getAffineTransform(pts1,pts2)
x_out = np.dot(M, [0,0,1]) - np.dot(M, [columnas,0,1])
y_out = np.dot(M, [0,0,1]) - np.dot(M, [columnas,0,1])
dst = cv2.warpAffine(img,M, (1300, 300), flags=cv2.INTER_CUBIC)

plt.figure(figsize=(20, 10))
plt.subplot(121),plt.imshow(img),plt.title('Input')
plt.subplot(122),plt.imshow(dst),plt.title('Output')
plt.show()

## Transformación de perspectiva

La transformación de perspectiva es ligeramente mas compleja que la transformación afín, en este caso podremos a traves de una matriz de 3x3 "transformar" una imágen de 3d a 2d.

En este caso la matriz tiene la siguiente forma:
![Imagen](https://scilab.io/wp-content/uploads/2017/08/persp-matrix-300x237.png)

En el ejemplo a continuación usaremos la transformación perspectiva para obtener una visión frontal de un objeto en una escena cualquiera.

In [ ]:
%pylab notebook

img = io.imread("resources/session4/libro.jpg")

def onclick(event):
    collector.append([event.xdata, event.ydata])


fig, ax = plt.subplots()
collector = []
cid = fig.canvas.mpl_connect('button_press_event', onclick)

plt.imshow(img)



In [ ]:
%matplotlib inline
print(collector)
pts1 = np.float32(collector[0:4])
pts2 = np.float32([[0,0],[300,0],[0,500],[300,500]])
M = cv2.getPerspectiveTransform(pts1,pts2)

dst = cv2.warpPerspective(img,M,(300,500))

plt.subplot(121)
plt.imshow(img)
plt.subplot(122)
plt.imshow(dst)


## Caso Práctico 1 - Aumentar número de muestras
Uno de los usos más comunes de la transformaciones afines es usarlas para generar nuevas muestras en nuestro dataset a partir de ligeras transformaciones de imagenes conocidas, por ejemplo rotaciones, traslaciones, etc...

En este ejemplo proponemos una función que nos permite generar tantas imágenes como necesitemos.

In [ ]:
def transform_image(img,ang_range,shear_range,trans_range):
    '''
    Esta funcion genera imágenes a partir de una sola imagen. Una distribución random uniforme 
    se usa para generar diferentes parameters para la transformación.
    
    Parameters
    ----------
    img: np.array(2d)
        Imagen inicial
    ang_range: int
        Rango de los ángulos para la rotación
    shear_range: int
        Rango de los valores usado para aplicar transformaciones afínes.
    trans_range: int
        Rango de los valores usados para aplicar translaciones
    '''
    
    # Rotación
    ang_rot = np.random.uniform(ang_range)-ang_range/2
    rows,cols,ch = img.shape    
    Rot_M = cv2.getRotationMatrix2D((cols/2,rows/2),ang_rot,1)

    # Translación
    tr_x = trans_range*np.random.uniform()-trans_range/2
    tr_y = trans_range*np.random.uniform()-trans_range/2
    Trans_M = np.float32([[1,0,tr_x],[0,1,tr_y]])

    # Shear
    pts1 = np.float32([[5,5],[20,5],[5,20]])

    pt1 = 5+shear_range*np.random.uniform()-shear_range/2
    pt2 = 20+shear_range*np.random.uniform()-shear_range/2
    
    
    pts2 = np.float32([[pt1,5],[pt2,pt1],[5,pt2]])

    shear_M = cv2.getAffineTransform(pts1,pts2)
        
    img = cv2.warpAffine(img,Rot_M,(cols,rows))
    img = cv2.warpAffine(img,Trans_M,(cols,rows))
    img = cv2.warpAffine(img,shear_M,(cols,rows))
    
    
    return img

In [ ]:
image= io.imread("resources/session2/our_face_dataset/test/10.jpg")

plt.figure(figsize=(12,12))
for i in range(100):
    img = transform_image(image,20,10,5)
    plt.subplot(10,10,i+1)
    plt.imshow(img)
    plt.axis('off')

plt.show()

# Caso Práctico 2 - Eugenizer

En este caso implementamos un puequeño ejemplo dónde se muestra como podemos transformar una imágen para que coincida con la perspectiva de una escena. En este caso cogeremos una imagen y la transforameros usando los landmarks detectados con el detector que entrenamos en la sessión anterior.

In [ ]:
from PIL import Image # Más facil manejar los PNG

gafas = Image.open("resources/session4/gafas2.png")
puntos_gafas = [[20, 85], [200, 85], [280, 85], [485, 85]]
plt.imshow(gafas)

In [ ]:
facedetector = dlib.get_frontal_face_detector()
facelandmark = dlib.shape_predictor('resources/session3/shape_predictor_68_face_landmarks.dat')
facerembedder = dlib.face_recognition_model_v1("resources/session3/dlib_face_recognition_resnet_model_v1.dat")

# Datos de entrada
input_video = cv2.VideoCapture('resources/session3/test.mp4')

# Escojemos un frame al azar
n_frames_used = np.random.randint(3200)
for i in range(n_frames_used):
    ret, frame = input_video.read()
    
# Lo convertimos a RGB
dlib_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
frame = Image.fromarray(frame)
frame = frame.convert('RGBA')
# Detectamos las caras
bboxes = facedetector(dlib_frame, 1)
print(len(bboxes))
# Iteramos sobre las caras encontradas
for k, bbox in enumerate(bboxes):
    
    # Detectamos los landmarks
    shape = facelandmark(dlib_frame, bbox)
    
    # Calculamos el angulo con los landmarks
    
    dX = shape.part(45).x - shape.part(36).x 
    dY = shape.part(45).y - shape.part(36).y 
    
    angle = np.rad2deg(np.arctan2(dY, dX))
    print("Size: ", dX, shape.part(41).y - shape.part(37).y)
    gafas_tmp = gafas.resize((dX + 10, shape.part(41).y - shape.part(37).y  + 10))
    gafas_tmp = gafas_tmp.rotate(-angle, expand=True)
    frame.paste(gafas_tmp, (shape.part(37).x - 5, shape.part(39).y - 5), gafas_tmp)
    
# Mostramos la imágen original
plt.figure(figsize=(50, 50));
plt.imshow(frame);

## Case 3 - Panorama
Para finalizar mostramos otra aplicación donde las trasformaciones afines se usan para poder generar panoramas usando múltiples imagenes. En este caso el procedimiento se basa en:

* Buscar puntos de interés en las imágenes.
* Correlacionar estos puntos.
* Usar las correspondéncias para generar una transformación afín que nos permita transformar estos puntos de una escena a la otra.
* Por ultimo transformar una de las imágenes con la matriz que hemos calculado previamente y combinar las dos imágenes para obtener el panorama.

In [ ]:
imageA = cv2.imread("resources/session4/panorama1.jpg")
imageB = cv2.imread("resources/session4/panorama2.jpg")

plt.figure(figsize=(20, 10))
plt.subplot("121")
plt.imshow(imageA)
plt.subplot("122")
plt.imshow(imageB)

In [ ]:
def detectAndDescribe(image):
    """
    Extraer features y puntos de interés en una imágen.
    
    Parameters
    ----------
    image : np.array(2d)
        Imagen que queremos procesar.
    
    Returns
    ---------
    (lst, lst)
        Puntos de interés y features para la imagen.
    """
    # Convertir la imagen a grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Usar el descriptor ORB
    orb = cv2.ORB_create()
    kps = orb.detect(gray)

    # Extraer features de la imagen
    (kps, features) = orb.compute(gray, kps)

    return (kps, features)

In [ ]:
(kpsA, featuresA) = detectAndDescribe(imageA)
(kpsB, featuresB) = detectAndDescribe(imageB)


In [ ]:
# Dibujamos los keypoints que hemos encontrado 
img1 = cv2.drawKeypoints(imageA, kpsA, None, color=(0,255,0), flags=0)
img2 = cv2.drawKeypoints(imageB, kpsB, None, color=(0,255,0), flags=0)
plt.figure(figsize=(20, 10))
plt.subplot("121")
plt.imshow(img1)
plt.subplot("122")
plt.imshow(img2)

In [ ]:
matcher = cv2.DescriptorMatcher_create("BruteForce")
rawMatches = matcher.knnMatch(featuresA, featuresB, 2)
matches = []

# Este pequeño fragmento de código elimina los outliers
for m in rawMatches:
    # Lowe's ratio test
    if len(m) == 2 and m[0].distance < m[1].distance * 0.75:
        matches.append((m[0].trainIdx, m[0].queryIdx))

# Convierte los Keypoints(OpenCV) a valores flotantes (np.array)
kpsA = np.float32([kp.pt for kp in kpsA])
kpsB = np.float32([kp.pt for kp in kpsB])

In [ ]:
#  Nos aseguramos que disponemos de un mínimo de 4 puntos para calcular la homografia A.K.A Transformación afín.
if len(matches) > 4:
    # Construimos los sets de puntos
    ptsA = np.float32([kpsA[i] for (_, i) in matches])
    ptsB = np.float32([kpsB[i] for (i, _) in matches])

    # Computamos la homografia
    (H, status) = cv2.findHomography(ptsA, ptsB, cv2.RANSAC,4.0)


In [ ]:
# Aplicamos la transformación y solapamos las imágenes
result = cv2.warpPerspective(imageA, H, (imageB.shape[1] + imageA.shape[1], imageB.shape[0]))
result[0:imageB.shape[0], 0:imageB.shape[1]] = imageB
result = result[0:,0:400,:]

# Mostramos el resultado
plt.figure(figsize=(20, 10))
plt.subplot("121")
plt.imshow(img1)
plt.subplot("122")
plt.imshow(result)